

## Two Sum

<https://leetcode.com/problems/two-sum/description/>

给定一个整数数列，找出其中和为特定值的那两个数。

你可以假设每个输入都只会有一种答案，同样的元素不能被重用。

示例:

```
Given nums = [2, 7, 11, 15], target = 9,

Because nums[0] + nums[1] = 2 + 7 = 9,
return [0, 1].
```

###  暴力算法

最直接的方法就是暴力算法。遍历所有的和，算法复杂度是 $O(n^2)$

In [14]:
class Solution:
    def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        l = len(nums)
        for i in range(l):
            for j in range(i+1, l):
                if nums[i] + nums[j] == target:
                    return [i, j]
                
Solution().twoSum([2, 7, 11, 15], 9)

[0, 1]


###  使用字典 （hash table）

观察一下 `[2, 7, 11, 15]`， 可以直接计算其解为 `[7, 2, ...]` ，存储 解和当前索引到字典中。 在往后的遍历中，先看看当前的数在不在解集合中，如果在，则直接返回之前的索引和当前的索引。如果不在，保存解和当前索引。

只需一次遍历即可，算法时间 $O(n)$

In [16]:
class Solution:
    def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        solution = dict()
        for i, n in enumerate(nums):
            if n in solution:
                return [solution[n], i]
            else:
                solution[target - n] = i

%time Solution().twoSum([2, 7, 11, 15], 9)

CPU times: user 16 µs, sys: 1 µs, total: 17 µs
Wall time: 24.1 µs


[0, 1]

## Add Two Numbers

两个链表分别表示两个数字，从低位指向高位的方向。求和

```
Input: (2 -> 4 -> 3) + (5 -> 6 -> 4)
Output: 7 -> 0 -> 8
Explanation: 342 + 465 = 807.
```

咋看很简单啊。 注意一些细节。 这里返回一个新的链表

In [1]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def addTwoNumbers(self, l1, l2):
        """
        :type l1: ListNode
        :type l2: ListNode
        :rtype: ListNode
        """
        
        x1 = l1
        x2 = l2
        carry = 0
        head = prev = ListNode(0)
        # loop1
        while (x1 and x2):
            val = x1.val + x2.val + carry
            prev.next = ListNode(val % 10)
            carry = val // 10
            prev = prev.next
            x1 = x1.next
            x2 = x2.next
            
        rx = x1 if x2 is None else x2
        
        # loop2
        while rx:
            val = rx.val + carry
            prev.next = ListNode(val % 10)
            carry = val // 10
            prev = prev.next
            rx = rx.next
            
        if carry:
            prev.next = ListNode(carry)
        
        return head.next
            

In [25]:
def create_list_node(num):
    head = prev = ListNode(0)
    while num:
        prev.next = ListNode(num % 10)
        num = num // 10
        prev = prev.next
    return head.next

def parse_list_node(ln):
    num = 0
    base = 1
    while ln:
        num += ln.val * base
        base = base * 10
        ln = ln.next
        
    return num 

def print_list_node(ln):
    l = list()
    while ln:
        l.append(ln.val)
        ln = ln.next
    print(l)
    
print_list_node(create_list_node(100))

print(parse_list_node(create_list_node(1024)))
assert parse_list_node(create_list_node(100)) == 100

def test(a, b):
    # create
    l1 = create_list_node(a)
    l2 = create_list_node(b)
    s = Solution().addTwoNumbers(l1, l2)
#     print_list_node(s)
    assert parse_list_node(s) == (a + b)
    
test(123, 3452323)

import random

for _ in range(100):
    a = random.randint(0, 100000000)
    b = random.randint(0, 100000000)
    test(a, b)
    

[0, 0, 1]
1024


In [24]:
random.randint(0, 100000000)

4353